In [7]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [8]:
# Class switching example 4
# Example of class switching controlled by a reducible Markov chain
# In this variant the job remains either in class 2 or class 3 forever
model = Network('reducible_cs')

In [9]:
# Block 1: nodes
node = np.empty(3, dtype=object)
node[0] = Delay(model, 'Queue 0')
node[1] = Delay(model, 'Queue 1')
node[2] = Delay(model, 'Queue 2')

# Block 2: classes
jobclass = np.empty(3, dtype=object)
jobclass[0] = ClosedClass(model, 'Class1', 1, node[0])
jobclass[1] = ClosedClass(model, 'Class2', 0, node[0])
jobclass[2] = ClosedClass(model, 'Class3', 0, node[0])

In [10]:
# Service configurations
node[0].set_service(jobclass[0], Exp.fit_mean(1.000000))  # (Queue 0,Class1)
node[0].set_service(jobclass[1], Exp.fit_mean(1.000000))  # (Queue 0,Class2)
node[0].set_service(jobclass[2], Exp.fit_mean(1.000000))  # (Queue 0,Class3)
node[1].set_service(jobclass[1], Exp.fit_mean(1.000000))  # (Queue 1,Class2)
node[2].set_service(jobclass[2], Exp.fit_mean(1.000000))  # (Queue 2,Class3)

In [ ]:
# Routing matrix with class switching - transient class example
P = model.init_routing_matrix()  # initialize routing matrix

# Class 1 routing: transitions to classes 2 and 3, or stays in class 1
P.set(jobclass[0], jobclass[0], node[0], node[0], 0.2)  # Stay in class 1 at node 0
P.set(jobclass[0], jobclass[1], node[0], node[0], 0.3)  # Switch to class 2 at node 0  
P.set(jobclass[0], jobclass[2], node[0], node[0], 0.5)  # Switch to class 3 at node 0

# Class 2 routing: once in class 2, stays in class 2
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)  # Class 2: node 0 to node 1
P.set(jobclass[1], jobclass[1], node[1], node[0], 1.0)  # Class 2: node 1 to node 0

# Class 3 routing: once in class 3, stays in class 3  
P.set(jobclass[2], jobclass[2], node[0], node[2], 1.0)  # Class 3: node 0 to node 2
P.set(jobclass[2], jobclass[2], node[2], node[0], 1.0)  # Class 3: node 2 to node 0

model.link(P)

In [12]:
#model.printRoutingMatrix()

In [ ]:
# Solve using JMT solver (better for class switching models)
solver = JMT(model)
AvgTable = solver.avg_chain_table()